<a target="_blank" href="https://colab.research.google.com/github/vishalbakshi/passage-spanner/blob/main/2024_12_18_PassageSpanner.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [49]:
from dataclasses import dataclass
from itertools import combinations
from fastcore.utils import L, groupby
from fastcore.test import test_eq
from regex import finditer, escape

In [50]:
document = """The quick brown fox jumps over the lazy dog.
The quick brown fox runs through the field.
A lazy dog sleeps in the sun."""

passages = [
    "quick brown fox",
    "lazy dog",
    "this passage is not in the document"
]
document, passages

('The quick brown fox jumps over the lazy dog.\nThe quick brown fox runs through the field.\nA lazy dog sleeps in the sun.',
 ['quick brown fox', 'lazy dog', 'this passage is not in the document'])

## Passage

The main object that I'm working with is a `Passage` which holds four important pieces of information:

- `text`: the passage text
- `start_pos`: the integer start position of the passage in the document
- `length`: the passage length
- `end_pos`: the integer end position of the passage in the document

In [51]:
passage = passages[0]
passage

'quick brown fox'

In [52]:
document.find(passage)

4

In [53]:
len(passage)

15

In [54]:
document.find(passage) + len(passage) - 1

18

In [55]:
@dataclass
class Passage:
    text: str
    start_pos: int
    length: int

    @property
    def end_pos(self):
        return self.start_pos if self.length == 0 else self.start_pos + self.length - 1

In [56]:
p = Passage(passage, document.find(passage), len(passage))
p

Passage(text='quick brown fox', start_pos=4, length=15)

In [57]:
def _passage(text, start_pos, length): return Passage(text=text, start_pos=start_pos, length=length)

In [58]:
p = _passage(passage, document.find(passage), len(passage))
p

Passage(text='quick brown fox', start_pos=4, length=15)

## Span

The next core object that I'm working with is a span which holds three pieces of information:

- `start` Passage
- `end` Passage
- `distance` between the two (distance = the difference between the first character of the ending Passage and the last character of the starting Passage)

In [59]:
start = p
start

Passage(text='quick brown fox', start_pos=4, length=15)

In [60]:
passage = passages[1]
passage

'lazy dog'

In [61]:
end = _passage(passage, document.find(passage), len(passage))
end

Passage(text='lazy dog', start_pos=35, length=8)

In [62]:
distance = end.start_pos - start.end_pos
distance

17

In [63]:
@dataclass
class Span:
    start: Passage
    end: Passage

    @property
    def distance(self): return self.end.start_pos - self.start.end_pos

In [64]:
s = Span(start, end)
s, s.distance

(Span(start=Passage(text='quick brown fox', start_pos=4, length=15), end=Passage(text='lazy dog', start_pos=35, length=8)),
 17)

In [65]:
def _span(start, end): return Span(start, end)

In [66]:
s = _span(start, end)
s, s.distance

(Span(start=Passage(text='quick brown fox', start_pos=4, length=15), end=Passage(text='lazy dog', start_pos=35, length=8)),
 17)

## Converting a list of passages to Passages

In [67]:
document

'The quick brown fox jumps over the lazy dog.\nThe quick brown fox runs through the field.\nA lazy dog sleeps in the sun.'

In [68]:
passages

['quick brown fox', 'lazy dog', 'this passage is not in the document']

A passage may occur multiple times in a document as this example shows, we want to capture all occurences as a unique Passage.

In [69]:
passage = passages[0]
passage

'quick brown fox'

In [70]:
L(finditer(escape(passage), document))

(#2) [<regex.Match object; span=(4, 19), match='quick brown fox'>,<regex.Match object; span=(49, 64), match='quick brown fox'>]

In [71]:
def _find(passage, document): return L(finditer(escape(passage), document))

In [72]:
ms = _find(passage, document)
ms

(#2) [<regex.Match object; span=(4, 19), match='quick brown fox'>,<regex.Match object; span=(49, 64), match='quick brown fox'>]

I want to extract from each map the text, start index and length of text:

In [73]:
m = ms[0]
m

<regex.Match object; span=(4, 19), match='quick brown fox'>

In [74]:
(m.group(), m.start(), len(m.group()))

('quick brown fox', 4, 15)

In [75]:
def _parsematch(m): return (m.group(), m.start(), len(m.group()))

In [76]:
_parsematch(m)

('quick brown fox', 4, 15)

Putting it all together for multiple passages with multiple occurences in the document:

In [77]:
L(passages).map(_find, document=document)

(#3) [[<regex.Match object; span=(4, 19), match='quick brown fox'>, <regex.Match object; span=(49, 64), match='quick brown fox'>],[<regex.Match object; span=(35, 43), match='lazy dog'>, <regex.Match object; span=(91, 99), match='lazy dog'>],[]]

The empty list is for the passage that was not in the document, and will go away when using `concat`.

In [78]:
passages[-1], _find(passages[-1], document)

('this passage is not in the document', (#0) [])

In [79]:
L(passages).map(_find, document=document).concat()

(#4) [<regex.Match object; span=(4, 19), match='quick brown fox'>,<regex.Match object; span=(49, 64), match='quick brown fox'>,<regex.Match object; span=(35, 43), match='lazy dog'>,<regex.Match object; span=(91, 99), match='lazy dog'>]

In [80]:
L(passages).map(_find, document=document).concat().map(_parsematch)

(#4) [('quick brown fox', 4, 15),('quick brown fox', 49, 15),('lazy dog', 35, 8),('lazy dog', 91, 8)]

In [81]:
L(passages).map(_find, document=document).concat().map(_parsematch).starmap(_passage)

(#4) [Passage(text='quick brown fox', start_pos=4, length=15),Passage(text='quick brown fox', start_pos=49, length=15),Passage(text='lazy dog', start_pos=35, length=8),Passage(text='lazy dog', start_pos=91, length=8)]

Lastly, I want to sort by start position (ascending).

In [82]:
ps = L(passages).map(_find, document=document).concat().map(_parsematch).starmap(_passage).sorted(key=lambda p: p.start_pos)
ps

(#4) [Passage(text='quick brown fox', start_pos=4, length=15),Passage(text='lazy dog', start_pos=35, length=8),Passage(text='quick brown fox', start_pos=49, length=15),Passage(text='lazy dog', start_pos=91, length=8)]

## Creating Spans from all Passage combinations

The document may contain any span of two Passages, so we want all of them.

In [83]:
combs = L(combinations(ps, 2))
combs

(#6) [(Passage(text='quick brown fox', start_pos=4, length=15), Passage(text='lazy dog', start_pos=35, length=8)),(Passage(text='quick brown fox', start_pos=4, length=15), Passage(text='quick brown fox', start_pos=49, length=15)),(Passage(text='quick brown fox', start_pos=4, length=15), Passage(text='lazy dog', start_pos=91, length=8)),(Passage(text='lazy dog', start_pos=35, length=8), Passage(text='quick brown fox', start_pos=49, length=15)),(Passage(text='lazy dog', start_pos=35, length=8), Passage(text='lazy dog', start_pos=91, length=8)),(Passage(text='quick brown fox', start_pos=49, length=15), Passage(text='lazy dog', start_pos=91, length=8))]

In [84]:
combs[0]

(Passage(text='quick brown fox', start_pos=4, length=15),
 Passage(text='lazy dog', start_pos=35, length=8))

In [85]:
combs.starmap(_span)

(#6) [Span(start=Passage(text='quick brown fox', start_pos=4, length=15), end=Passage(text='lazy dog', start_pos=35, length=8)),Span(start=Passage(text='quick brown fox', start_pos=4, length=15), end=Passage(text='quick brown fox', start_pos=49, length=15)),Span(start=Passage(text='quick brown fox', start_pos=4, length=15), end=Passage(text='lazy dog', start_pos=91, length=8)),Span(start=Passage(text='lazy dog', start_pos=35, length=8), end=Passage(text='quick brown fox', start_pos=49, length=15)),Span(start=Passage(text='lazy dog', start_pos=35, length=8), end=Passage(text='lazy dog', start_pos=91, length=8)),Span(start=Passage(text='quick brown fox', start_pos=49, length=15), end=Passage(text='lazy dog', start_pos=91, length=8))]

I only want to keep Spans that are less than or equal to `max_dist`

In [86]:
max_dist=20
spans = combs.starmap(_span).filter(lambda s: s.distance <= max_dist)
spans

(#2) [Span(start=Passage(text='quick brown fox', start_pos=4, length=15), end=Passage(text='lazy dog', start_pos=35, length=8)),Span(start=Passage(text='lazy dog', start_pos=35, length=8), end=Passage(text='quick brown fox', start_pos=49, length=15))]

## Storing the longest Spans

Each Passage may start one or more Spans. I want the longest Span for each starting Passage.

In [87]:
gs = groupby(spans, lambda s: s.start.start_pos)
gs

{4: [Span(start=Passage(text='quick brown fox', start_pos=4, length=15), end=Passage(text='lazy dog', start_pos=35, length=8))],
 35: [Span(start=Passage(text='lazy dog', start_pos=35, length=8), end=Passage(text='quick brown fox', start_pos=49, length=15))]}

There is only valid Span per starting position under `max_dist`.

In [88]:
L(max(g, key=lambda s: s.end.end_pos) for g in gs.values())

(#2) [Span(start=Passage(text='quick brown fox', start_pos=4, length=15), end=Passage(text='lazy dog', start_pos=35, length=8)),Span(start=Passage(text='lazy dog', start_pos=35, length=8), end=Passage(text='quick brown fox', start_pos=49, length=15))]

For illustrative purposes, I'll show my `max` operation on all Spans (even those greater than `max_dist`)

In [89]:
gs2 = groupby(combs.starmap(_span), lambda s: s.start.start_pos)
gs2

{4: [Span(start=Passage(text='quick brown fox', start_pos=4, length=15), end=Passage(text='lazy dog', start_pos=35, length=8)),
  Span(start=Passage(text='quick brown fox', start_pos=4, length=15), end=Passage(text='quick brown fox', start_pos=49, length=15)),
  Span(start=Passage(text='quick brown fox', start_pos=4, length=15), end=Passage(text='lazy dog', start_pos=91, length=8))],
 35: [Span(start=Passage(text='lazy dog', start_pos=35, length=8), end=Passage(text='quick brown fox', start_pos=49, length=15)),
  Span(start=Passage(text='lazy dog', start_pos=35, length=8), end=Passage(text='lazy dog', start_pos=91, length=8))],
 49: [Span(start=Passage(text='quick brown fox', start_pos=49, length=15), end=Passage(text='lazy dog', start_pos=91, length=8))]}

In [90]:
L(max(g, key=lambda s: s.end.end_pos) for g in gs2.values())

(#3) [Span(start=Passage(text='quick brown fox', start_pos=4, length=15), end=Passage(text='lazy dog', start_pos=91, length=8)),Span(start=Passage(text='lazy dog', start_pos=35, length=8), end=Passage(text='lazy dog', start_pos=91, length=8)),Span(start=Passage(text='quick brown fox', start_pos=49, length=15), end=Passage(text='lazy dog', start_pos=91, length=8))]

Wrapping this in a function:

In [91]:
def longest(spans):
    gs = groupby(spans, lambda s: s.start.start_pos)
    return L(max(g, key=lambda s: s.end.end_pos) for g in gs.values())

In [92]:
longest(spans)

(#2) [Span(start=Passage(text='quick brown fox', start_pos=4, length=15), end=Passage(text='lazy dog', start_pos=35, length=8)),Span(start=Passage(text='lazy dog', start_pos=35, length=8), end=Passage(text='quick brown fox', start_pos=49, length=15))]

## Testing out functions

In [93]:
@dataclass
class Passage:
    text: str
    start_pos: int
    length: int

    @property
    def end_pos(self):
        return self.start_pos if self.length == 0 else self.start_pos + self.length - 1

@dataclass
class Span:
    start: Passage
    end: Passage

    @property
    def distance(self): return self.end.start_pos - self.start.end_pos

In [94]:
def _parsematch(m): return (m.group(), m.start(), len(m.group()))
def _find(passage, document): return L(finditer(escape(passage), document))

def _passage(text, start_pos, length): return Passage(text=text, start_pos=start_pos, length=length)
def _span(start, end): return Span(start, end)

def longest(spans):
    gs = groupby(spans, lambda s: s.start.start_pos)
    return L(max(g, key=lambda s: s.end.end_pos) for g in gs.values())

def _text(doc, passages, max_dist=20):
    ps = L(passages).map(_find, document=doc).concat().map(_parsematch).starmap(_passage).sorted(key=lambda p: p.start_pos)
    combs = L(combinations(ps, 2))
    spans = longest(combs.starmap(_span).filter(lambda s: s.distance <= max_dist))
    return L(doc[s.start.start_pos:s.end.end_pos+1] for s in spans)

### Overlapping passages

In [95]:
document = """The quick brown fox jumps over the lazy dog.
A clever fox leaps across the sleeping hound.
The nimble creature bounds through the garden."""

passages = [
    "brown fox jumps over",
    "fox jumps over the lazy",
    "over the lazy dog.\nA clever",
    "the lazy dog",
    "sleeping hound.\nThe nimble"
]

expected = L(['brown fox jumps over the lazy dog.\nA clever','fox jumps over the lazy dog.\nA clever','over the lazy dog'])
document, passages

('The quick brown fox jumps over the lazy dog.\nA clever fox leaps across the sleeping hound.\nThe nimble creature bounds through the garden.',
 ['brown fox jumps over',
  'fox jumps over the lazy',
  'over the lazy dog.\nA clever',
  'the lazy dog',
  'sleeping hound.\nThe nimble'])

In [96]:
res = _text(document, passages)
test_eq(res, expected)

In [97]:
document = """Once upon a time, a magical crystal glowed brightly.
Its light danced across the ancient cavern walls.
Deep within, secrets waited to be discovered."""

passages = [
    "magical crystal",           # first in chain
    "crystal glowed",           # overlaps with previous
    "glowed brightly",          # overlaps with previous
    "brightly.\nIts light",     # overlaps with previous, crosses line
    "light danced",             # overlaps with previous
    "Deep within",              # separate passage
    "to be discovered"          # separate passage
]


expected = L([
    'magical crystal glowed brightly.\nIts light',
    'crystal glowed brightly.\nIts light danced',
    'glowed brightly.\nIts light danced','brightly.\nIts light danced',
    'Deep within, secrets waited to be discovered'
    ])
document, passages

('Once upon a time, a magical crystal glowed brightly.\nIts light danced across the ancient cavern walls.\nDeep within, secrets waited to be discovered.',
 ['magical crystal',
  'crystal glowed',
  'glowed brightly',
  'brightly.\nIts light',
  'light danced',
  'Deep within',
  'to be discovered'])

In [98]:
res = _text(document, passages)
test_eq(res, expected)

### Repeated passages

In [99]:
document = """The quick brown fox jumps over the lazy dog.
The quick brown fox runs through the field.
A lazy dog sleeps in the sun."""

passages = [
    "quick brown fox",  # Appears twice
    "lazy dog"         # Appears twice
]

expected = L(['quick brown fox jumps over the lazy dog','lazy dog.\nThe quick brown fox'])
document, passages

('The quick brown fox jumps over the lazy dog.\nThe quick brown fox runs through the field.\nA lazy dog sleeps in the sun.',
 ['quick brown fox', 'lazy dog'])

In [100]:
res = _text(document, passages)
test_eq(res, expected)

### Empty passages

In [101]:
document = "doc"
passages = [""]
expected = L(['doc','oc','c'])
document, passages

('doc', [''])

In [102]:
res = _text(document, passages)
test_eq(res, expected)

### Passages with whitespace characters

In [103]:
document = "This is a\nsample\tdocument with some content."
passages = [" ", "   ", "\n", "\t"]
expected = L([' is a\nsample\t',' a\nsample\tdocument ','\nsample\tdocument ','\tdocument with some ',' with some ',' some '])
document, passages

('This is a\nsample\tdocument with some content.', [' ', '   ', '\n', '\t'])

In [104]:
res = _text(document, passages)
test_eq(res, expected)

### Passages with special characters or formatting

In [105]:
document = """Special *formatting* and (punctuation) test!
Some text with @#$% special chars.
More text with <html> tags and line-breaks
    plus some indentation."""
passages = [
    "*formatting*",
    "@#$%",
    "<html>",
    "    plus"
]

expected = L([])
document, passages

('Special *formatting* and (punctuation) test!\nSome text with @#$% special chars.\nMore text with <html> tags and line-breaks\n    plus some indentation.',
 ['*formatting*', '@#$%', '<html>', '    plus'])

In [106]:
res = _text(document, passages)
test_eq(res, expected)

### Non-existent passages

In [107]:
document = """The sky is blue.
The grass is green.
The sun is yellow.
The rose is red.
The cloud is white."""

passages = [
    "The sky is blue.",
    "The grass is green.",
    "This passage is not in the document.",
    "The sun is yellow.",
    "The rose is red.",
    "The cloud is white.",
    "This passage is also not in the document."
]

expected = L(["The sky is blue.\nThe grass is green.",
"The grass is green.\nThe sun is yellow.",
"The sun is yellow.\nThe rose is red.\nThe cloud is white.",
"The rose is red.\nThe cloud is white."])
document, passages

('The sky is blue.\nThe grass is green.\nThe sun is yellow.\nThe rose is red.\nThe cloud is white.',
 ['The sky is blue.',
  'The grass is green.',
  'This passage is not in the document.',
  'The sun is yellow.',
  'The rose is red.',
  'The cloud is white.',
  'This passage is also not in the document.'])

In [108]:
res = _text(document, passages)
test_eq(res, expected)